# Convert Audio Files to MP3 Format

Convert audio files that failed in batch processing due to unsupported formats.

**Problem:** Whisper API rejected files with these formats:
- Raw `.aac` files
- `.amr` (AMR-NB) files  
- `.mp4` files (3GPP/AAC codec)
- `.mp3` files that are actually AAC codec

**Solution:** Convert all failed files to true MP3 format using ffmpeg.

**Strategy:**
1. Read failed files list from batch processing CSV
2. Convert each file to MP3 with quality settings: 128kbps, 44.1kHz
3. Save as `original_name_converted.mp3` (preserves originals)
4. Validate conversions
5. Generate summary report

In [ ]:
import os
import subprocess
from pathlib import Path
import pandas as pd
from src.voice_eval.config import load_config

In [ ]:
# Load configuration
audio_dir = load_config('input', 'audio_dir')
reports_dir = load_config('output', 'reports_dir')

print(f"Audio directory: {audio_dir}")
print(f"Reports directory: {reports_dir}")

## Check FFmpeg Installation

In [ ]:
# Check if ffmpeg is installed
try:
    result = subprocess.run(['ffmpeg', '-version'], capture_output=True, text=True, timeout=5)
    if result.returncode == 0:
        version_line = result.stdout.split('\n')[0]
        print(f"✓ FFmpeg found: {version_line}")
    else:
        print("✗ FFmpeg not working properly")
        print("Please install ffmpeg: brew install ffmpeg (macOS) or apt-get install ffmpeg (Linux)")
except FileNotFoundError:
    print("✗ FFmpeg not found")
    print("Please install ffmpeg: brew install ffmpeg (macOS) or apt-get install ffmpeg (Linux)")
except Exception as e:
    print(f"✗ Error checking ffmpeg: {e}")

## Load Failed Files from Previous Batch

In [ ]:
# Read the latest batch processing summary
summary_csv = f"{reports_dir}/batch_whisper_gpt4o_summary_latest.csv"
df = pd.read_csv(summary_csv)

# Filter failed files
failed_files = df[df['status'] == 'failed']['file'].tolist()

print(f"Found {len(failed_files)} failed files to convert:")
print(f"\nFile format breakdown:")
format_counts = {}
for f in failed_files:
    ext = Path(f).suffix.lower()
    format_counts[ext] = format_counts.get(ext, 0) + 1

for ext, count in sorted(format_counts.items()):
    print(f"  {ext}: {count} files")

print(f"\nFirst 5 failed files:")
for f in failed_files[:5]:
    print(f"  - {f}")
print(f"  ... and {len(failed_files) - 5} more")

## Conversion Function

In [ ]:
def convert_to_mp3(
    input_path: str,
    output_dir: str = None,
    bitrate: str = "128k",
    sample_rate: int = 44100
) -> dict:
    """
    Convert audio file to MP3 format using ffmpeg.
    
    Args:
        input_path: Path to input audio file
        output_dir: Directory for output (default: same as input)
        bitrate: MP3 bitrate (default: 128k)
        sample_rate: Sample rate in Hz (default: 44100)
    
    Returns:
        dict with status, output_path, and error (if any)
    """
    input_path = Path(input_path)
    
    if output_dir is None:
        output_dir = input_path.parent
    else:
        output_dir = Path(output_dir)
        output_dir.mkdir(parents=True, exist_ok=True)
    
    # Generate output filename: originalname_converted.mp3
    output_filename = f"{input_path.stem}_converted.mp3"
    output_path = output_dir / output_filename
    
    result = {
        "input_file": input_path.name,
        "output_file": output_filename,
        "status": "failed",
        "output_path": None,
        "error": None
    }
    
    try:
        # FFmpeg command:
        # -i: input file
        # -vn: no video (audio only)
        # -ar: audio sample rate
        # -ab: audio bitrate
        # -f mp3: force MP3 format
        # -y: overwrite output file
        cmd = [
            'ffmpeg',
            '-i', str(input_path),
            '-vn',  # No video
            '-ar', str(sample_rate),
            '-ab', bitrate,
            '-f', 'mp3',
            '-y',  # Overwrite
            str(output_path)
        ]
        
        # Run ffmpeg (suppress output unless error)
        process = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            timeout=300  # 5 minute timeout
        )
        
        if process.returncode == 0 and output_path.exists():
            # Verify the output file is not empty
            file_size = output_path.stat().st_size
            if file_size > 0:
                result.update({
                    "status": "success",
                    "output_path": str(output_path),
                    "file_size_mb": file_size / (1024 * 1024)
                })
            else:
                result["error"] = "Output file is empty"
        else:
            result["error"] = f"FFmpeg failed with return code {process.returncode}\n{process.stderr[:200]}"
    
    except subprocess.TimeoutExpired:
        result["error"] = "Conversion timed out (>5 minutes)"
    except Exception as e:
        result["error"] = str(e)
    
    return result

print("✓ Conversion function defined")

## Batch Conversion

In [ ]:
print("="*80)
print("STARTING BATCH CONVERSION")
print("="*80)
print(f"Files to convert: {len(failed_files)}")
print(f"Output format: MP3 (128kbps, 44.1kHz)")
print(f"Output location: {audio_dir}/")
print()

conversion_results = []

for i, filename in enumerate(failed_files, 1):
    input_path = Path(audio_dir) / filename
    
    print(f"[{i}/{len(failed_files)}] Converting: {filename}")
    
    # Check if input file exists
    if not input_path.exists():
        print(f"  ✗ Input file not found: {input_path}")
        conversion_results.append({
            "input_file": filename,
            "status": "failed",
            "error": "Input file not found"
        })
        continue
    
    # Convert
    result = convert_to_mp3(input_path, output_dir=audio_dir)
    conversion_results.append(result)
    
    if result["status"] == "success":
        print(f"  ✓ Converted to: {result['output_file']} ({result['file_size_mb']:.2f} MB)")
    else:
        print(f"  ✗ Failed: {result['error']}")

print("\n" + "="*80)
print("CONVERSION COMPLETE")
print("="*80)

## Summary Report

In [ ]:
# Calculate statistics
total_files = len(conversion_results)
successful = [r for r in conversion_results if r.get("status") == "success"]
failed = [r for r in conversion_results if r.get("status") == "failed"]

total_size_mb = sum(r.get("file_size_mb", 0) for r in successful)

print(f"\n📊 CONVERSION SUMMARY")
print("="*80)
print(f"\nFiles Converted:")
print(f"  ✓ Successful: {len(successful)} ({len(successful)/total_files*100:.1f}%)")
print(f"  ✗ Failed:     {len(failed)} ({len(failed)/total_files*100:.1f}%)")
print(f"  📁 Total:      {total_files}")

if successful:
    print(f"\nOutput Files:")
    print(f"  Total size: {total_size_mb:.2f} MB")
    print(f"  Average size: {total_size_mb/len(successful):.2f} MB per file")
    print(f"  Location: {audio_dir}/")
    print(f"  Naming pattern: [original_name]_converted.mp3")

if failed:
    print(f"\n❌ FAILED CONVERSIONS ({len(failed)}):")
    print("-" * 80)
    for r in failed:
        print(f"  • {r['input_file']}")
        print(f"    Error: {r.get('error', 'Unknown error')}")

print(f"\n✅ Next Step: Re-run batch processing (notebook 06) to transcribe converted files")
print(f"   Set SKIP_EXISTING=True to avoid reprocessing successful files")

## Save Conversion Report

In [ ]:
# Save conversion results to CSV
from datetime import datetime

conversion_df = pd.DataFrame(conversion_results)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

csv_path_timestamped = f"{reports_dir}/audio_conversion_summary_{timestamp}.csv"
csv_path_latest = f"{reports_dir}/audio_conversion_summary_latest.csv"

conversion_df.to_csv(csv_path_timestamped, index=False)
conversion_df.to_csv(csv_path_latest, index=False)

print(f"\n💾 Conversion report saved:")
print(f"  • {csv_path_timestamped}")
print(f"  • {csv_path_latest}")

# Display summary table
print(f"\n📋 Conversion Results (first 10 rows):")
print(conversion_df.head(10).to_string())

## Create File List for Batch Processing

Generate a list of successfully converted files to help with next steps.

In [ ]:
if successful:
    converted_files = [r['output_file'] for r in successful]
    
    print(f"\n📝 Successfully Converted Files ({len(converted_files)}):")
    print("-" * 80)
    for f in converted_files[:10]:
        print(f"  {f}")
    if len(converted_files) > 10:
        print(f"  ... and {len(converted_files) - 10} more")
    
    print(f"\n💡 These files are now ready for batch processing with Whisper API")